In [1]:
# -----------------------------------------------
# 1️⃣ Import Libraries
# -----------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# -----------------------------------------------
# 2️⃣ Load Cleaned Energy Data
# -----------------------------------------------
DATA_DIR = Path.cwd().parent / 'data'
energy_path = DATA_DIR / 'energy_data_transformed.csv'

print("Loading cleaned energy data...")
energy_df = pd.read_csv(energy_path)

# Convert date column to datetime
energy_df['date'] = pd.to_datetime(energy_df['date'])
print("✅ Data loaded successfully!")
print(energy_df.head())

# -----------------------------------------------
# 3️⃣ Sort and Group Data
# -----------------------------------------------
energy_df = energy_df.sort_values(['CONS_NO', 'date'])
grouped = energy_df.groupby('CONS_NO')

# -----------------------------------------------
# 4️⃣ Create Time-based Features
# -----------------------------------------------
# Compute rolling features per consumer
energy_df['rolling_mean_3'] = grouped['consumption'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
energy_df['rolling_std_3'] = grouped['consumption'].transform(lambda x: x.rolling(window=3, min_periods=1).std())
energy_df['pct_change'] = grouped['consumption'].transform(lambda x: x.pct_change().fillna(0))

# Lag features (previous day’s consumption)
energy_df['lag_1'] = grouped['consumption'].shift(1)
energy_df['lag_2'] = grouped['consumption'].shift(2)

# Fill missing values after shift
energy_df = energy_df.fillna(0)

# -----------------------------------------------
# 5️⃣ Statistical Features (Consumer Level)
# -----------------------------------------------
consumer_stats = energy_df.groupby('CONS_NO')['consumption'].agg(['mean', 'std', 'max', 'min']).reset_index()
consumer_stats.rename(columns={'mean':'mean_usage', 'std':'usage_std', 'max':'max_usage', 'min':'min_usage'}, inplace=True)

energy_df = pd.merge(energy_df, consumer_stats, on='CONS_NO', how='left')

# -----------------------------------------------
# 6️⃣ Derived Features
# -----------------------------------------------
energy_df['z_score'] = (energy_df['consumption'] - energy_df['mean_usage']) / (energy_df['usage_std'] + 1e-6)
energy_df['sudden_spike'] = (energy_df['pct_change'] > 0.5).astype(int)
energy_df['sudden_drop'] = (energy_df['pct_change'] < -0.5).astype(int)

# -----------------------------------------------
# 7️⃣ Anomaly Indicator (Preliminary)
# -----------------------------------------------
# Mark potential anomalies (this will later be refined by ML)
energy_df['potential_anomaly'] = ((abs(energy_df['z_score']) > 3) | (energy_df['sudden_spike'] == 1) | (energy_df['sudden_drop'] == 1)).astype(int)

print("\n✅ Feature engineering complete!")
print(energy_df.head(10))

# -----------------------------------------------
# 8️⃣ Visualization
# -----------------------------------------------
plt.figure(figsize=(12, 6))
sns.histplot(energy_df['consumption'], bins=50, kde=True)
plt.title('Distribution of Energy Consumption (After Feature Engineering)')
plt.show()

plt.figure(figsize=(12, 6))
sns.scatterplot(x='date', y='consumption', hue='potential_anomaly', data=energy_df.sample(2000))
plt.title('Sample of Consumption with Potential Anomalies')
plt.xlabel('Date')
plt.ylabel('Consumption')
plt.show()

# -----------------------------------------------
# 9️⃣ Save Engineered Dataset
# -----------------------------------------------
output_path = DATA_DIR / 'energy_features.csv'
energy_df.to_csv(output_path, index=False)
print(f"\n💾 Engineered features saved to: {output_path}")


Loading cleaned energy data...


FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/energy_theft_and_oil_leak/data/energy_data_transformed.csv'